In [ ]:
#Importing a series of packages used throughout the pipeline
import GEOparse
import pandas as pd
import numpy as np
import os
import json
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.decomposition import PCA

In [ ]:
#The series accession id for the study you are analyzing
geo_accession_id = "GSE54917"
#Establishing a working directory
WORKDIR = '/Users/MaayanLab/Desktop/Allie'
#Separating control and treated samples
control_samples = ['GSM1326549', 'GSM1326550', 'GSM1326551']
treated_samples = ['GSM1326552', 'GSM1326553', 'GSM1326554', 'GSM1326555']

In [ ]:
#Creating a dictionary of assigned control and treated samples
def merge(control_samples, treated_samples): 
    res = {**control_samples, **treated_samples} 
    return res 
      
control_samples = { i : 'control' for i in control_samples }
treated_samples = { i : 'treated' for i in treated_samples }
all_samples = merge(control_samples, treated_samples) 
print(all_samples) 

In [ ]:
#Parse the GEO data using the Accession ID
gse = GEOparse.get_GEO(geo=geo_accession_id, destdir="./")

In [ ]:
#Create a list of samples to use in the development of the expression matrix
list_samples = list(all_samples.keys())
list_samples

In [ ]:
#Visualization of expression matrix
pivoted_samples = gse.pivot_samples('VALUE')[list_samples]
pivoted_samples.head()

In [ ]:
#Determine the total amount of probes used in the study
pivoted_samples_average = pivoted_samples.median(axis=1)
print("Number of probes before filtering: ", len(pivoted_samples_average))

In [ ]:
#Filtering out unexpressed probes
expression_threshold = pivoted_samples_average.quantile(0.3)
expressed_probes = pivoted_samples_average[pivoted_samples_average >= expression_threshold].index.tolist()
print("number of probes above threshold: ", len(expressed_probes))

In [ ]:
#Redefine expression data using only the expressed probes
exprsdata = gse.pivot_samples("VALUE").loc[expressed_probes]
exprsdata = exprsdata.T
#Deletes additional samples that aren't being analyzed
exprsdata = exprsdata[exprsdata.index.isin(list_samples)]
#Drop any probe columns where expression data is missing or negative
exprsdata.dropna(axis = 1)

In [ ]:
#Quantile normalization of data
rank_mean = exprsdata.stack().groupby(exprsdata.rank(method='first').stack().astype(int)).mean()
exprsdata.rank(method='min').stack().astype(int).map(rank_mean).unstack()

In [ ]:
#Compute PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(exprsdata)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3'])

In [ ]:
#Making Dataframe of samples to concatenate with principal components
samplesDf = pd.DataFrame.from_dict(all_samples, orient = 'index', columns = ['type'])
samplesDf.reset_index(inplace=True)

In [ ]:
#Concatenate sample data with PCA data
principalDf = pd.concat([samplesDf, principalDf], axis=1)
principalDf

In [ ]:
#PCA scatter plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize = (12,12))
ax = fig.gca(projection='3d')
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel('Principal Component 3', fontsize = 15)
ax.set_title('3 Component PCA', fontsize = 20)

types = ('control', 'treated')
colors = ['green', 'violet']
for type, color in zip(types, colors):
    indicesToKeep = principalDf['type'] == type
    ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1'], 
               principalDf.loc[indicesToKeep, 'principal component 2'], principalDf.loc[indicesToKeep, 'principal component 3'], c = color, s = 50)
ax.legend(types)

In [ ]:
#Calculate variance ratio
pca.explained_variance_ratio_

In [ ]:
#Transpose data matrix for sorting, index correlated to probe IDs
exprsdata = exprsdata.T
exprsdata

In [ ]:
#Sort expression matrix using 800 genes with greatest variance
variances = np.var(exprsdata, axis=1)
srt_idx = variances.argsort()[::-1]
data_sub = exprsdata.iloc[srt_idx].iloc[:800]
data_sub.index = data_sub.index.map(str)
data_sub

In [ ]:
#Extract probe ids from data
probeids = list(data_sub.index)
probeids

In [ ]:
#Upload annotation file as dictionary
dict1 = {}
with open(WORKDIR + 'probe2gene.txt') as f:
    for line in f:
        line = line.strip()
        (platform, probe, symbol) = line.split()
        dict1[probe] = symbol

In [ ]:
#Examine how many ids are duplicates for gene symbols/unmatched
len(set(probeids) - dict1.keys())

In [ ]:
#Reset index and replace with gene symbols, view as dataframe
exprsdata = pd.DataFrame(exprsdata)
exprsdata['symbol'] = exprsdata.index.to_series().map(dict1)
exprsdata.reset_index(inplace=True)
data = exprsdata.set_index('symbol')
#Drop probe id column
data = data.drop('ID_REF', axis=1)
data

In [ ]:
#Drop rows that aren't associated with a particular gene symbol
data = data.reset_index().dropna().set_index('symbol')
data

In [ ]:
#Standardized data to a text file
data_file = (WORKDIR + 'expression_matrix_top800_genes.txt')
data.to_csv(data_file, sep='\t')
data_file

In [ ]:
#Post expression matrix to clustergrammer
import requests, json
clustergrammer_url = 'http://amp.pharm.mssm.edu/clustergrammer/matrix_upload/'

r = requests.post(clustergrammer_url, files={'file': open(data_file, 'rb')})
link = r.text
link

In [ ]:
#Display clustergram
from IPython.display import IFrame
display(IFrame(link, width="1000", height="1000"))

In [ ]:
#Import required packages for characteristic direction and utilize warning statements
import warnings
from scipy.stats import chi2
from scipy.stats.mstats import zscore
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [ ]:
#Define characteristic direction function
def chdir(data, sampleclass, genes, gamma=1., sort=True, calculate_sig=False, nnull=10, sig_only=False, norm_vector=True):
    data.astype(float)
    #sampleclass = np.array(map(int, sampleclass))
    
    m_non0 = sampleclass != 0
    m1 = sampleclass[m_non0] == 1
    m2 = sampleclass[m_non0] == 2
    
    data = data[:, m_non0]
    data = zscore(data)
    
    n1 = m1.sum() # number of controls
    n2 = m2.sum() # number of experiments
    
    meanvec = data[:,m2].mean(axis=1) - data[:,m1].mean(axis=1) 
    
    pca = PCA(n_components=None)
    pca.fit(np.array(data.T))
    
    cumsum = pca.explained_variance_ratio_
    keepPC = len(cumsum[cumsum > 0.001])
    
    v = pca.components_[0:keepPC].T
    r = pca.transform(data.T)[:,0:keepPC]
    dd = ( np.dot(r[m1].T,r[m1]) + np.dot(r[m2].T,r[m2]) ) / float(n1+n2-2)
    sigma = np.mean(np.diag(dd))
    
    shrunkMats = np.linalg.inv(gamma*dd + sigma*(1-gamma)*np.eye(keepPC))
    b = np.dot(v, np.dot(np.dot(v.T, meanvec), shrunkMats))
    
    if norm_vector:
        b /= np.linalg.norm(b)
        
    grouped = zip([abs(item) for item in b],b,genes)
    if sort:
        grouped = sorted(grouped,key=lambda x: x[0], reverse=True)
    
    if not calculate_sig: # return sorted b and genes.
        res = [(item[1],item[2]) for item in grouped]
        return res
    else: # generate a null distribution of chdirs
        nu = n1 + n2 - 2
        y1 = np.random.multivariate_normal(np.zeros(keepPC), dd, nnull).T * np.sqrt(nu / chi2.rvs(nu,size=nnull))
        y2 = np.random.multivariate_normal(np.zeros(keepPC), dd, nnull).T * np.sqrt(nu / chi2.rvs(nu,size=nnull))
        y = y2 - y1
        
        nullchdirs = []
        for col in y.T:
            bn = np.dot(np.dot(np.dot(v,shrunkMats), v.T), np.dot(col,v.T))
            bn /= np.linalg.norm(bn)
            bn = bn ** 2
            bn.sort()
            bn = bn[::-1] ## sort in decending order
            nullchdirs.append(bn)

        nullchdirs = np.array(nullchdirs).T
        nullchdirs = nullchdirs.mean(axis=1)
        b_s = b ** 2 
        b_s.sort()
        b_s = b_s[::-1] # sorted b in decending order
        relerr = b_s / nullchdirs ## relative error
        # ratio_to_null
        ratios = np.cumsum(relerr)/np.sum(relerr)- np.linspace(1./len(meanvec),1,len(meanvec))
        res = [(item[1],item[2], ratio) for item, ratio in zip(grouped, ratios)] 
        print('Number of significant genes: %s'%(np.argmax(ratios)+1))
        if sig_only:
            return res[0:np.argmax(ratios)+1]
        else:
            return res

In [ ]:
#Make sample classes, ensure that there is a distinction between control/treated samples
data_cd = {}

sample_classes = {}
sample_class = np.zeros(data.shape[1], dtype=np.int32)
sample_class[samplesDf['type'].values == 'control'] = 1
sample_class[samplesDf['type'].values == 'treated'] = 2
sample_classes = sample_class

print(sample_classes)

In [ ]:
#CD results
cd_res = chdir(data.values, sample_classes, data.index, gamma=.5, sort=False, calculate_sig=False)
cd_coefs = np.array(list(map(lambda x: x[0], cd_res)))

srt_idx = np.abs(cd_coefs).argsort()[::-1]
cd_coefs = cd_coefs[srt_idx][:600]
sorted_DEGs = data.index[srt_idx][:600]
up_genes = dict(zip(sorted_DEGs[cd_coefs > 0], cd_coefs[cd_coefs > 0]))
dn_genes = dict(zip(sorted_DEGs[cd_coefs < 0], cd_coefs[cd_coefs < 0]))
data_cd['up'] = up_genes
data_cd['dn'] = dn_genes

In [ ]:
#Retrieve up and down gene sets
up_list = list(up_genes.keys())
dn_list = list(dn_genes.keys())

In [ ]:
#Post data to Enrichr
import json

ENRICHR_URL = 'https://amp.pharm.mssm.edu/Enrichr'

def _enrichr_add_list(genes, meta=''):
    genes_str = '\n'.join(genes)
    payload = {
        'list': (None, genes_str),
        'description': (None, meta)
    }
    # POST genes to the /addList endpoint
    response = requests.post("%s/addList" % ENRICHR_URL, files=payload)
    list_ids = json.loads(response.text)
    return list_ids

def enrichr_link(genes, meta=''):
    list_ids = _enrichr_add_list(genes, meta)
    shortId = list_ids['shortId']
    link = '%s/enrich?dataset=%s' % (ENRICHR_URL, shortId)
    return link

In [ ]:
#Print links for further analysis
for key, d in data_cd.items():
    time.sleep(1)
    genes = list(data_cd[key].keys())
    genes = [str(g) for g in genes]
    link = enrichr_link(genes, key)
    print(key)
    print(link)